In [24]:
import geopandas as gpd
import pandas as pd

In [18]:
maui = gpd.read_file('./data/mauiStructureTypes.json')
fire = gpd.read_file('./data/EOS')
fire = fire.loc[fire.ID == 'F7']
parcels = gpd.read_file('/data/Parcels_-_Maui_County.zip')
damage = gpd.read_file('./data/InferencedBuildingDamage.zip')

In [16]:
# damaged buildings
gpd.sjoin(damage.to_crs(maui.crs), 
          maui, 
          predicate='intersects', 
          how='left')[['ClassLabel','geometry']].reset_index()\
.to_file(
              './cleanedData/DamagedBuildings.json',
              driver='GeoJSON')

In [23]:
fire.boundary.to_file('./cleanedData/FireBoundary.json', driver='GeoJSON')

In [38]:
df = pd.read_csv('./cleanedData/BuildingDamage & USA Structures_InferencedBuildingDamage.csv')
df = df.loc[df['Occ Cls']=='Residential']
df.head(1).T

,0
Build Id,358935.0
Censuscode,15009031404.0
Class Label,Undamaged
Fips,15009.0
Image Date,7/18/2014 12:00:00 AM
Image Name,1030010035A81000
Occ Cls,Residential
Outbldg,NaN
Prim Occ,Multi - Family Dwelling
Prod Date,6/3/2019 12:00:00 AM


In [31]:
df.columns

Index(['Build Id', 'Censuscode', 'Class Label', 'Fips', 'Image Date',
       'Image Name', 'Occ Cls', 'Outbldg', 'Prim Occ', 'Prod Date',
       'Prop Addr', 'Prop City', 'Prop St', 'Prop Zip', 'Remarks', 'Sec Occ',
       'Source', 'Usng', 'Uuid', 'Val Method', 'Centroid X', 'Centroid Y',
       'Confidence', 'Geometry', 'Geometry (HI Structures)', 'H Adj Elev',
       'Height', 'L Adj Elev', 'Latitude', 'Longitude', 'Shape Area',
       'Shape Length', 'Sqfeet', 'Sqft', 'Sqmeters'],
      dtype='object')

In [35]:
damage.columns

Index(['SQFT', 'centroid_x', 'centroid_y', 'ClassLabel', 'confidence',
       'geometry'],
      dtype='object')

In [45]:
gpd.GeoDataFrame(df.merge(damage, 
         right_on=['centroid_x', 'centroid_y'], 
         left_on=['Centroid X', 'Centroid Y'])\
[['Class Label', 'geometry',]]).reset_index()\
    .to_crs('WGS84').to_file(
    './cleanedData/DamagedResidences.json',
    driver='GeoJSON'
)

In [59]:
damagedResiParcels = gpd.read_file('./cleanedData/DamagedResiParcels')
damagedResiParcels = damagedResiParcels.loc[(damagedResiParcels.damaged==1)&(damagedResiParcels.prop_type!='Non-residential'),
                                            ['prop_type', 'geometry']].reset_index()
damagedResiParcels.to_file(
    './cleanedData/DamagedResidentialParcels.json',
    driver='GeoJSON'
)

In [56]:
damagedResiParcels.columns

Index(['tmk_txt', 'damaged', 'yr_built', 'effec_yr', 'bldng_val', 'full_baths',
       'half_baths', 'bedrooms', 'rooms', 'sf_living', 'sf_bldng', 'pittcode',
       'prop_type', 'geometry'],
      dtype='object')

In [54]:
damagedResiParcels.prop_type.unique()

array(['Non-residential', 'Short-term Rental', 'Owner-Occupied/Homeowner',
       'Long-term Rental', 'Non-Owner-Occupied/Residential',
       'Commercialized Residential', 'Apartment', 'Multiple Codes',
       'Timeshare'], dtype=object)

In [8]:
pitts = dict(zip(
    sorted(list(parcels.pittcode.unique())),
    ['Timeshare', 'Non-Owner-Occupied/Residential',
    'Apartment', 'Non-residential',
    'Non-residential', 'Non-residential',
    'Non-residential', 'Non-residential',
    'Owner-Occupied/Homeowner', 'Multiple Codes', 
    'Commercialized Residential', 'Short-term Rental', 
    'Long-term Rental']
))
fireParcels = gpd.overlay(fire, parcels, how='intersection')
fireParcels['pittcode'] = fireParcels['pittcode'].map(lambda x: pitts[x])